In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime as dt
import statistics as stat
import csv

In [2]:
asset_value = [860690000000, 15350000000, 5850000000, 74430000000, 15370000000, 12160000000, 786970000,
              72640000000, 269050000000, 237470000, 238750000, 36110000, 24550000000, 957340000, 680370000, 8400000000]
total_value = sum(asset_value)
w_eq = [(i/total_value) for i in asset_value]

In [3]:
init_investment = 1000000
price = pd.read_csv("data_adjclose.csv", index_col=0)
price.index = pd.to_datetime(price.index)
tickers = [i for i in price]
price["Index Value"] = 0
price.iat[0, 16] = init_investment

price.head()

,VTI,VGK,VPL,VWO,PFF,IEI,BWX,AGG,BND,FXY,FXF,UDN,IAU,AOA,AOK,USO,Index Value
Date,,,,,,,,,,,,,,,,,
2014-01-02,84.378860,46.217941,51.057976,33.389236,25.745323,108.694878,27.300598,90.548790,67.201050,93.250000,108.667809,26.176607,11.88,37.321964,27.329165,273.839996,1000000
2014-01-03,84.361145,46.225983,51.244526,33.347179,25.898182,108.613411,27.233980,90.582863,67.209480,93.220001,108.008736,26.050152,12.01,37.347317,27.380636,270.000000,0
2014-01-06,84.139259,46.266125,51.032539,33.027584,26.051073,108.785301,27.352930,90.701874,67.268166,93.779999,108.208450,26.137697,12.01,37.296593,27.346323,268.559998,0
2014-01-07,84.680595,46.579334,51.185162,33.162144,26.002424,108.875786,27.371956,90.744354,67.352066,93.419998,107.549370,26.079332,11.95,37.465664,27.432093,268.640015,0
2014-01-08,84.742729,46.555244,51.261486,33.103275,26.044107,108.468697,27.272049,90.472290,67.133980,93.250000,107.409569,26.020967,11.89,37.448750,27.397795,265.279999,0


In [4]:
#unit = init_investment* daily_weight / daily_price
weight =pd.DataFrame(columns=tickers, index=price.index)
weight.iloc[0,:] = w_eq

units = pd.DataFrame(columns=tickers, index=price.index)
unit = (init_investment*weight.iloc[0,:]) / price.iloc[0,:16]    #series
units.iloc[:,:] = unit.values

In [5]:
#weight = daily_price*dailyunits / daily_indexvalue

for i in range(1, price.shape[0]):
    new_index = np.dot(price.iloc[i,:16],units.iloc[i,:])
    price.iat[i,16] = new_index
    w = price.iloc[i,:16] * units.iloc[i,:] / new_index  #weight = rice*units / indexvalue
    weight.iloc[i,:] = w.values
    
weight.head()

,VTI,VGK,VPL,VWO,PFF,IEI,BWX,AGG,BND,FXY,FXF,UDN,IAU,AOA,AOK,USO
Date,,,,,,,,,,,,,,,,
2014-01-02,0.632197,0.0112749,0.00429696,0.0546706,0.0112896,0.0089318,0.000578048,0.0533558,0.197624,0.000174427,0.000175367,2.65236e-05,0.0180325,0.000703189,0.000499748,0.00617
2014-01-03,0.632045,0.0112765,0.00431253,0.0546,0.0113563,0.00892483,0.00057662,0.0533742,0.197642,0.000174366,0.000174298,2.63947e-05,0.0182293,0.000703645,0.000500673,0.00608329
2014-01-06,0.631582,0.0113078,0.00430286,0.0541797,0.0114451,0.00895597,0.00058024,0.0535461,0.198191,0.000175747,0.000174953,2.65338e-05,0.018264,0.000704027,0.000500998,0.00606236
2014-01-07,0.632769,0.0113329,0.0042962,0.0541542,0.011372,0.00892286,0.000578016,0.0533287,0.197541,0.00017428,0.000173101,2.63548e-05,0.0180905,0.000704018,0.000500295,0.00603673
2014-01-08,0.633624,0.011334,0.00430526,0.0540914,0.0113973,0.00889498,0.000576262,0.0532016,0.197022,0.00017407,0.000172982,2.6312e-05,0.0180108,0.000704133,0.000499977,0.0059649


### Equilibrium Portfolio 的 Volatility Control Index

In [6]:
performance = pd.DataFrame(columns=["Return", "Volatility", "Yearly Vol"], index=price.index)

for i in range(1, price.shape[0]):
    tmp_ret = (price.iloc[i,16] - price.iloc[i-1,16]) / price.iloc[i-1,16]
    performance.iat[i,0] = tmp_ret
    tmp_vol = performance.iloc[i-20:i,0].std()
    #print(tmp_vol)
    tmp_y_vol = tmp_vol * np.sqrt(252/20)
    performance.iat[i,1] = tmp_vol
    performance.iat[i,2] = tmp_y_vol

performance.tail()

,Return,Volatility,Yearly Vol
Date,,,
2019-04-24,-0.00104932,0.00352578,0.0125153
2019-04-25,-0.0013842,0.00345676,0.0122703
2019-04-26,0.00460927,0.00332692,0.0118094
2019-04-29,0.000609314,0.00337208,0.0119697
2019-04-30,0.000407483,0.00327075,0.01161


In [7]:
print(performance["Yearly Vol"].max())

0.04967093115150719


<b>由於Index的波動率不超過5%，因此不進行部位減持。